In [14]:
#IMPORT LIBRARIES
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pprint import pprint
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re
import time
import subprocess
from youtube_search import YoutubeSearch
#import scrapy


#KICKOFF URL KINO-ZEIT.DE CINEMA PROGRAM
url='https://www.kino-zeit.de/filme/filmstarts/aktuell'

#CHOOSING NUMBER OF UPCOMING WEEKS TO SCRAPE
print('How many upcoming weeks would you like to scrape at kino-zeit.de\n---')
weeks=int(input())



#FUNCTION TURN URL INTO HTML
def make_soup(url):
    html=requests.get(url).content
    return BeautifulSoup(html,'html.parser')

#FUNCTION CREATING THE URL TO FLIP TO NEXT WEEKS PAGE
def next_url(soup):
    url=''.join([f'https://www.kino-zeit.de/filme/filmstarts/{i["href"]}' for i in soup.find_all("a",{"class": "next btn btn-primary"})])
    return url

#FUNCTION FLIPPING THROUGH THE WEEKLY OVERVIEWS
#Picking film titles, start dates and the film's individual page
def how_much_soup(url, weeks):
    df_loop=pd.DataFrame(columns=['start','film','director','actor','link','image','distro','country','trailer'])
    lst_start=[]
    lst_film=[]
    lst_link=[]
    for x in range(weeks): #for the number of weeks we are browsing
        
        soup= make_soup(url)
        lst_film.append([i.text for i in soup.find_all(['h3','a'],{'class': 'filmlink'})])
        lst_start.append([i.text for i in soup.find_all('div',{'class':'col-md-2 infospalte-1'})])
        lst_start=re.findall(r'\d\d.\d\d.\d\d\d\d', str(lst_start))
        lst_link.append([f'https://www.kino-zeit.de{i["href"]}' for i in soup.find_all(["h3","a"],{"class": "filmlink"})])
        url=next_url(soup) #calling the function for the url we flip to for the following week
    
    lst_film=[i for sublst in lst_film for i in sublst]
    df_loop['film']=lst_film
    df_loop['start']=lst_start
    lst_link= [i for sublst in lst_link for i in sublst]
    df_loop['link']=lst_link
    return (df_loop)


#CREATING OVERVIEW
df=pd.DataFrame
lst_start=[]
lst_film=[]
lst_link=[]
regie_list=[]
darsteller_list=[]
list_img_url=[]
list_verleih=[]
list_prodland=[]

#STARTIN PROGRAM
df=how_much_soup(url, weeks)

#BROWSING THE FILMS' INDIVIDUAL PAGES FOR FURTHER DETAILS 
for i in df['link']:
    sub_page=make_soup(i)
    regie=[i.text.replace('\nRegie','').replace('\n\n','').split(',') for i in sub_page.find_all('div',{'class':'field field--name-field-regie field--type-entity-reference field--label-above'})]
    regie_list.append(regie[0])
    darsteller=[i.text.replace('\nDarsteller','').split('\n') for i in sub_page.find_all('div',{'class':'field field--name-field-hauptdarsteller field--type-entity-reference field--label-above'})]
    darsteller_list.append(darsteller)
    img_url=[i['data-big'].split('?') for i in sub_page.find_all('img',{'id':'headerbildbig'})]
    list_img_url.append(i[0] for i in img_url)
    verleih=[i.text.replace('\nFilmverleih','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-distribution-film field--type-entity-reference field--label-above'})]
    list_verleih.append(verleih)
    prodland=[i.text.replace('\nProduktionsland','').replace('\n\n','') for i in sub_page.find_all(['div','a'],{'class':'field field--name-field-produktionsland field--type-entity-reference field--label-above'})]
    list_prodland.append(prodland)
    
#FILLING THE DATAFRAME WITH THOSE NEW DATA
df['director']=regie_list
df['actor']=darsteller_list
df['image']=list_img_url
df['distro']=list_verleih
df['country']=list_prodland


#CLEAN ACTORS COLUMN
for i in range(len(df.actor)):
    if len(df['actor'].loc[i])==2:
        df['actor'].loc[i]=df['actor'].loc[i][0]
        df['actor'].loc[i]=list(filter(lambda x: x, list(df['actor'].loc[i])))

#TURN LISTS INTO STRINGS TO CLEAN THEM UP
for i in range(len(df)):
    df['film']=df['film'].str.replace('\(\d\d\d\d\)','')
    df['country'][i]=''.join(df['country'][i]).replace('\n',', ')
    df['director'][i]=', '.join(df['director'][i]).replace('\n',', ')
    df['actor'][i]=', '.join(df['actor'][i])
    df['image'][i]=''.join(df['image'][i])
    df['distro'][i]=''.join(df['distro'][i]).replace('\n',', ')
    
    
###YOUTUBE TRAILER
#FUNCTION SEARCHING YOUTUBE RETURNING A YOUTUBE LINK
def youtube_link (film, distro):
    results = YoutubeSearch(f'{film} {distro} Trailer', max_results=1).to_dict() #returns a dictionar
    if results ==[]:
        return('couldnt find youtube trailer')
    else:
        suffix = results[0]['url_suffix']
        youtube_link= f'www.youtube.com{suffix}'
        return(youtube_link)

df['trailer']='-'
for i in range(len(df)):
    df['trailer'].loc[i] = youtube_link(df['film'][i],df['distro'][i])

###CREATE FILES WITH TIMESTAMP CSV EXCEL
timestr = time.strftime("%Y%m%d")
df.to_csv(f'Filmstarts_{timestr}.csv')
df.to_excel(f'Filmstarts_{timestr}.xlsx')


#RINGS THE BELL WHEN SCRIPT IS DONE
audio_file = "service-bell.mp3"
return_code = subprocess.call(["afplay", audio_file])

#SHOW DATAFRAME
df

How many upcoming weeks would you like to scrape at kino-zeit.de
---
2


,start,film,director,actor,link,image,distro,country,trailer
0,01.10.2020,Contemporary Past - Die Gegenwart der Vergange...,Kamil Majchrzak,,https://www.kino-zeit.de/node/50404,https://www.kino-zeit.de/sites/default/files/s...,Les Funambules Filmproduktion,Polen,www.youtube.com/watch?v=-tgm0_y9M3M
1,01.10.2020,Die Misswahl - Der Beginn einer Revolution,Philippa Lowthorpe,"Keira Knightley, Gugu Mbatha-Raw, Jessie Buckl...",https://www.kino-zeit.de/node/48346,https://www.kino-zeit.de/sites/default/files/s...,"eOne , Paramount Pictures Germany GmbH",Großbritannien,www.youtube.com/watch?v=3Qqy6IeVum8
2,01.10.2020,Drachenreiter,Tomer Eshed,,https://www.kino-zeit.de/node/49005,https://www.kino-zeit.de/sites/default/files/s...,Constantin Filmverleih,Deutschland,www.youtube.com/watch?v=1BT7g6wcGhs
3,01.10.2020,Enfant Terrible,Oskar Roehler,"Oliver Masucci, Katja Riemann, Hary Prinz, Ant...",https://www.kino-zeit.de/node/48025,https://www.kino-zeit.de/sites/default/files/s...,Weltkino Filmverleih,,www.youtube.com/watch?v=taggXKKa0RA
4,01.10.2020,Good Kisser,Wendy Jo Carlton,"Kari Alison Hodge, Julia Eringer, Courtney McC...",https://www.kino-zeit.de/node/50833,https://www.kino-zeit.de/sites/default/files/s...,Pro-Fun Media Filmverleih,USA,www.youtube.com/watch?v=paHdPYlf21A
5,01.10.2020,"Gott, du kannst ein Arsch sein!",André Erkau,"Til Schweiger, Heike Makatsch, Benno Fürmann, ...",https://www.kino-zeit.de/node/50476,https://www.kino-zeit.de/sites/default/files/s...,Leonine,Deutschland,www.youtube.com/watch?v=WQHiwRNvSiE
6,01.10.2020,Hurtigruten und Norwegen,Volker Wischnowski,,https://www.kino-zeit.de/node/50852,https://www.kino-zeit.de/sites/default/files/s...,imFilm Agentur + Verleih,Deutschland,www.youtube.com/watch?v=2NmadshVF14
7,01.10.2020,Jim Knopf und die wilde 13,Dennis Gansel,"Henning Baum, Michael Bully Herbig, Solomon Go...",https://www.kino-zeit.de/node/48472,https://www.kino-zeit.de/sites/default/files/s...,Warner Bros. Entertainment GmbH,Deutschland,www.youtube.com/watch?v=JwG3kZSox2g
8,01.10.2020,Niemals Selten Manchmal Immer,Eliza Hittman,"Talia Ryder, Sidney Flanigan, Théodore Pelleri...",https://www.kino-zeit.de/node/48347,https://www.kino-zeit.de/sites/default/files/s...,Universal Pictures International Germany GmbH,"USA, Großbritannien",www.youtube.com/watch?v=tdz_Si1IffM
9,02.10.2020,On the Rocks,Sofia Coppola,"Rashida Jones, Bill Murray, Marlon Wayans, Jen...",https://www.kino-zeit.de/node/50572,https://www.kino-zeit.de/sites/default/files/s...,"A24, Trafalgar Releasing",USA,www.youtube.com/watch?v=Xn3sK4WiviA
